In [ ]:
import numpy as np
import pandas as pd
df = pd.read_parquet("fraud-cleaned-sample.parquet")

# Train/test split

We're using time-series data, so we'll split based on time.

In [ ]:
first = df['timestamp'].min()
last = df['timestamp'].max()
cutoff = first + ((last - first) * 0.7)

df = df.copy()

train = df[df['timestamp'] <= cutoff]
test = df[df['timestamp'] > cutoff]

In [ ]:
import cloudpickle as cp
feature_pipeline = cp.load(open('feature_pipeline.sav', 'rb'))

# Train the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection

rfc = RandomForestClassifier(n_estimators=16, max_depth=8, random_state=404, class_weight="balanced_subsample")

svecs = feature_pipeline.fit_transform(train)
rfc.fit(svecs, train["label"])


In [ ]:
from sklearn.metrics import classification_report

predictions = rfc.predict(feature_pipeline.fit_transform(test))
print(classification_report(test.label.values, predictions))


In [ ]:
from mlworkflows import plot
df, chart = plot.binary_confusion_matrix(test["label"], predictions)
chart

In [ ]:
df

# Save the trained model as a pipeline stage

In [ ]:
from mlworkflows import util
util.serialize_to(rfc, "rfc.sav")